In [1]:
import pathlib
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import anndata
from pybedtools import BedTool

## Parameter

In [2]:
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial/ITSpatial_rms_results_collapsed.tsv'
dmr_prefix = 'DGmCH'

black_list_path = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'
skip_quantile = 0.25
delta_to_mean = 0.1
dms_cutoff = 1

## Read DMR

In [3]:
# get column and n_sample
dmr_column = pd.read_csv(dmr_path, sep='\t', nrows=0).columns
n_sample = dmr_column.size - 6
dmr_column = dmr_column.map(lambda i: i[18:] if 'methylation_level_' in i else i)
samples = dmr_column[6:]

dmr_region_infos = []
mc_rates = []
hypo_sig_dict = {sample: [] for sample in samples}
hyper_sig_dict = {sample: [] for sample in samples}

dmr_df = pd.read_csv(dmr_path, sep='\t', skiprows=1, names=dmr_column)
print(dmr_df.shape[0], 'dmr before filter')
dmr_df.index = dmr_df.index.map(lambda i: f'{dmr_prefix}_{i}')

# save raw info
dmr_bed = dmr_df.iloc[:, :4].copy()
mc_rate = dmr_df.iloc[:, 6:].copy()

1588445 dmr before filter


## Save unfiltered dmr info

In [4]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    hdf['Rate'] = mc_rate
    hdf['Bed'] = dmr_bed

In [5]:
dmr_bed.reset_index().iloc[:, [1, 2, 3, 0]].to_csv(pathlib.Path(dmr_path).parent / 'TotalDMR.nofilter.bed', 
                                                   index=None, header=None, sep='\t')

## Filter DMR based on min-max delta and blacklist

In [7]:
# filter by min max delta first, if DMR do not pass min max filter, it won't have sig sample anyway
delta_judge = (mc_rate.max(axis=1) - mc_rate.min(axis=1)) > delta_to_mean
delta_index = mc_rate.index[delta_judge]

print(delta_index.size, 'dmr pass delta filter')

1444475 dmr pass delta filter


In [8]:
# filter by blacklist
black_list = pd.read_csv(black_list_path, sep='\t', header=None)
black_list = BedTool.from_dataframe(black_list)

_dmr_bed = BedTool.from_dataframe(dmr_bed.reset_index().iloc[:, [1,2,3,0]])
filtered_dmr_bed = _dmr_bed.intersect(black_list, v=True).to_dataframe().set_index('name')

white_index = filtered_dmr_bed.index
print(white_index.size, 'dmr pass blacklist filter')

1568410 dmr pass blacklist filter


In [9]:
dmr_df = dmr_df.loc[delta_index & white_index]
dmr_df = dmr_df[dmr_df['number_of_dms'] >= dms_cutoff]
print(dmr_df.shape[0], 'dmr after filter')
dmr_df.head()

1426526 dmr after filter


,#chr,start,end,number_of_dms,hypermethylated_samples,hypomethylated_samples,IT-L23+ACA,IT-L23+AI,IT-L23+MOp,IT-L4+MOs,...,IT-L6+AI,IT-L4+SSp,IT-L6+ACA,IT-L5+ACA,IT-L4+SSs,IT-L6+PFC,IT-L6+SSp,IT-L6+SSs,IT-L6+MOs,IT-L6+MOp
DGmCH_0,chr1,3006782,3006782,1,NaN,NaN,0.916667,0.857143,0.750000,0.750000,...,0.500000,0.461538,1.000000,0.894737,0.357143,0.714286,0.947368,0.916667,0.857143,0.900000
DGmCH_1,chr1,3012518,3012518,1,NaN,"IT-L23+MOp,IT-L23+MOs",0.893333,0.769231,0.765957,0.937500,...,0.894737,0.876712,0.946429,0.957895,0.933333,0.909091,0.933333,0.975610,0.957447,0.960526
DGmCH_2,chr1,3013472,3013579,3,"IT-L6+ACA,IT-L5+ACA,IT-L5+MOp","IT-L23+ORB,IT-L23+SSp,IT-L4+MOp,IT-L23+SSs,IT-...",0.833333,0.728571,0.709220,0.714286,...,0.761905,0.652482,0.974684,0.925170,0.672566,0.888889,0.860294,0.826087,0.941176,0.870229
DGmCH_3,chr1,3015124,3015124,1,NaN,"IT-L4+SSp,IT-L4+SSs",0.957447,0.941176,0.980000,0.952381,...,1.000000,0.725490,0.942857,0.980000,0.550000,0.960000,0.961538,0.918919,0.916667,0.913043
DGmCH_4,chr1,3027398,3027514,2,"IT-L5+SSp,IT-L6+ACA,IT-L5+ACA,IT-L6+PFC,IT-L6+MOp","IT-L23+MOp,IT-L23+ORB,IT-L23+SSp,IT-L23+SSs",0.615789,0.419355,0.484472,0.473684,...,0.607143,0.625806,0.764228,0.825758,0.608000,0.694737,0.700637,0.704082,0.670330,0.742857


## Calculate sample delta to robust mean

In [10]:
mc_rate = mc_rate.loc[dmr_df.index].copy()
n_sample = mc_rate.shape[1]

assert skip_quantile < 0.5
skip_n = min(1, round(n_sample * skip_quantile))

In [11]:
# ~1M rows / min, depending on n_sample, not very slow
def robust_mean(row):
    return np.nanmean(np.sort(row)[skip_n:-skip_n])
robust_mean = np.apply_along_axis(robust_mean, 1, mc_rate.values)
robust_mean = pd.Series(robust_mean, index=mc_rate.index)

In [12]:
delta = mc_rate - robust_mean[:, None]
# 0 is hypo DMR, 1 is hyper DMR, nan is not significant
judge = (delta > 0)[delta.abs() > delta_to_mean]

In [13]:
robust_mean_hypo_records = {}
robust_mean_hyper_records = {}

for sample, sample_col in judge.iteritems():
    hypo_dmr = sample_col[sample_col == 0].index
    hyper_dmr = sample_col[sample_col == 1].index
    
    robust_mean_hypo_records[sample] = hypo_dmr
    robust_mean_hyper_records[sample] = hyper_dmr

## Assign DMR to each sample

In [14]:
# parse results and add the robust mean judge
for sample in samples:
    # here saved a list of np.array
    hypo_index = dmr_df[dmr_df['hypomethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hyper_index = dmr_df[dmr_df['hypermethylated_samples'].fillna('').apply(lambda i: sample in i)].index
    hypo_sig_dict[sample] = hypo_index & robust_mean_hypo_records[sample]
    hyper_sig_dict[sample] = hyper_index & robust_mean_hyper_records[sample]
    print(sample, 'HypoDMR ', hypo_sig_dict[sample].size, sep='\t')
    print(sample, 'HyperDMR', hyper_sig_dict[sample].size, sep='\t')


IT-L23+ACA	HypoDMR 	212604
IT-L23+ACA	HyperDMR	146322
IT-L23+AI	HypoDMR 	159608
IT-L23+AI	HyperDMR	128180
IT-L23+MOp	HypoDMR 	386916
IT-L23+MOp	HyperDMR	99426
IT-L4+MOs	HypoDMR 	172133
IT-L4+MOs	HyperDMR	70249
IT-L5+SSp	HypoDMR 	221379
IT-L5+SSp	HyperDMR	140733
IT-L23+ORB	HypoDMR 	149947
IT-L23+ORB	HyperDMR	146175
IT-L23+PFC	HypoDMR 	191822
IT-L23+PFC	HyperDMR	238774
IT-L23+SSp	HypoDMR 	398916
IT-L23+SSp	HyperDMR	107375
IT-L23+MOs	HypoDMR 	335600
IT-L23+MOs	HyperDMR	94795
IT-L5+PFC	HypoDMR 	124418
IT-L5+PFC	HyperDMR	250070
IT-L5+AI	HypoDMR 	144248
IT-L5+AI	HyperDMR	127374
IT-L5+MOs	HypoDMR 	226538
IT-L5+MOs	HyperDMR	71022
IT-L4+MOp	HypoDMR 	276038
IT-L4+MOp	HyperDMR	147983
IT-L5+MOp	HypoDMR 	237796
IT-L5+MOp	HyperDMR	74728
IT-L5+ORB	HypoDMR 	101415
IT-L5+ORB	HyperDMR	101427
IT-L23+SSs	HypoDMR 	400733
IT-L23+SSs	HyperDMR	89653
IT-L5+SSs	HypoDMR 	171062
IT-L5+SSs	HyperDMR	94012
IT-L6+AI	HypoDMR 	54948
IT-L6+AI	HyperDMR	91206
IT-L4+SSp	HypoDMR 	395555
IT-L4+SSp	HyperDMR	183797
IT-L6+ACA	H

In [15]:
with pd.HDFStore(pathlib.Path(dmr_path).parent / 'DMRInfo.h5') as hdf:
    for sample, hypo_index in hypo_sig_dict.items():
        hdf[f'HypoDMR/{sample}'] = pd.Series(hypo_index)
    for sample, hyper_index in hyper_sig_dict.items():
        hdf[f'HyperDMR/{sample}'] = pd.Series(hyper_index)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23+ACA'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23+AI'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L23+MOp'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural nam

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L6+SSp'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L6+SSs'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: 'IT-L6+MOs'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural namin

## Dump DMR bed

In [18]:
hypo_dir = pathlib.Path(dmr_path).parent / 'HypoDMR'
hypo_dir.mkdir(exist_ok=True)
for sample, hypo_index in hypo_sig_dict.items():
    _bed = dmr_bed.loc[hypo_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hypo_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

In [19]:
hyper_dir = pathlib.Path(dmr_path).parent / 'HyperDMR'
hyper_dir.mkdir(exist_ok=True)
for sample, hyper_index in hyper_sig_dict.items():
    _bed = dmr_bed.loc[hyper_index].reset_index().iloc[:, [1, 2, 3, 0]].to_csv(
        hyper_dir / f'{sample}.DMS{dms_cutoff}.bed', sep='\t', index=None, header=None)

## DMR hits matrix

In [20]:
sig_dict = hypo_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hypo_dir / 'TotalHits.h5ad')


... storing '#chr' as categorical


In [21]:
sig_dict = hyper_sig_dict

rows = []
cols = []
datas = []
for i, (sample, dmr_index) in enumerate(sig_dict.items()):
    col = dmr_index.map(lambda i: i.split('_')[1]).astype(int).values
    row = (np.ones_like(col) * i).astype(int)
    data = np.ones_like(col)
    rows.append(row)
    cols.append(col)
    datas.append(data)
datas = np.concatenate(datas)
cols = np.concatenate(cols)
rows = np.concatenate(rows)
hits = csr_matrix((datas, (rows, cols)),
                  shape=(mc_rate.shape[1], dmr_bed.shape[0]))

# obs is DMR, var is sample, because all analysis is dmr focused
dmr_hits = anndata.AnnData(X=hits.T,
                           obs=dmr_bed,
                           var=pd.DataFrame([], index=mc_rate.columns))

dmr_hits.write_h5ad(hyper_dir / 'TotalHits.h5ad')

... storing '#chr' as categorical
